In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("IMDB.db")
cursor = conn.cursor()

In [1]:
from datetime import datetime

now = datetime.now()
print("Compilado em:", now.strftime("%d/%m/%Y %H:%M:%S"))

Compilado em: 16/10/2025 23:12:40


# IMDB Querys - LAB 13 (ME315)
## Analisando estrutura das tabelas presentes no banco de dados

**Tabela basics**:

In [20]:
head_basics = pd.read_sql_query("SELECT * FROM basics LIMIT 6;", conn)
print(head_basics)

      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short          Pauvre Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   
5  tt0000006     short       Chinese Opium Den       Chinese Opium Den   

   isAdult startYear endYear runtimeMinutes                    genres  
0      0.0      1894      \N              1         Documentary,Short  
1      0.0      1892      \N              5           Animation,Short  
2      0.0      1892      \N              5  Animation,Comedy,Romance  
3      0.0      1892      \N             12           Animation,Short  
4      0.0      1893      \N              1              Comedy,Short  
5      0.0      1894      \N              1      

**Tabela principals**:

In [21]:
head_principals = pd.read_sql_query("SELECT * FROM principals LIMIT 6;", conn)
print(head_principals)

      tconst  ordering     nconst         category                      job  \
0  tt0000001       1.0  nm1588970             self                       \N   
1  tt0000001       2.0  nm0005690         director                       \N   
2  tt0000001       3.0  nm0005690         producer                 producer   
3  tt0000001       4.0  nm0374658  cinematographer  director of photography   
4  tt0000002       1.0  nm0721526         director                       \N   
5  tt0000002       2.0  nm1335271         composer                       \N   

  characters  
0     [Self]  
1         \N  
2         \N  
3         \N  
4         \N  
5         \N  


**Tabela ratings**:

In [22]:
head_ratings = pd.read_sql_query("SELECT * FROM ratings LIMIT 6;", conn)
print(head_ratings)

      tconst  averageRating  numVotes
0  tt0000001            5.7    2092.0
1  tt0000002            5.6     283.0
2  tt0000003            6.5    2099.0
3  tt0000004            5.4     183.0
4  tt0000005            6.2    2835.0
5  tt0000006            5.0     196.0


# Quais são os 5 filmes com as maiores notas (averageRating)?

In [3]:
query = """
SELECT DISTINCT titletype
FROM basics
"""
teste = pd.read_sql_query(query,conn)
print(teste)

       titleType
0          short
1          movie
2        tvShort
3        tvMovie
4      tvEpisode
5       tvSeries
6   tvMiniSeries
7      tvSpecial
8          video
9      videoGame
10       tvPilot


In [6]:
query = """
SELECT b.primaryTitle, r.averageRating, r.numVotes
FROM ratings r
JOIN basics b ON r.tconst = b.tconst
WHERE b.titletype = 'movie'
ORDER BY r.averageRating DESC, r.numVotes DESC
LIMIT 5;
"""
filmes_melhores_avaliados = pd.read_sql_query(query, conn)
print(filmes_melhores_avaliados)


               primaryTitle  averageRating  numVotes
0                    Kaveri           10.0    1023.0
1                   Kurukku           10.0     451.0
2  Jedal Dar Omghe 30 Metri           10.0     142.0
3                 Sargashte           10.0     134.0
4           Gorgeous Rascal           10.0     115.0


Os 5 filmes melhor avaliados estão com nota 10. Sendo esses:

- **Kaveri**
- **Kurukku**
- **Jedal Dar Omghe 30 Metri**
- **Sargashte**
- **Gorgeous Rascal**


Porém, é perceptível que todos têm numero de avaliações baixas (tendo em vista que o IMDB é uma plataforma internacional). Logo, para uma seleção mais confiável, será utilizado um critério para evitar esse cenário.

In [8]:
query = """
SELECT b.primaryTitle, r.averageRating, r.numVotes
FROM ratings r
JOIN basics b ON r.tconst = b.tconst
WHERE b.titletype = 'movie'
AND r.numVotes > 100000
ORDER BY r.averageRating DESC, r.numVotes DESC
LIMIT 5;
"""
filmes_melhores_avaliados_com_criterio = pd.read_sql_query(query, conn)
print(filmes_melhores_avaliados_com_criterio)

                                    primaryTitle  averageRating   numVotes
0                       The Shawshank Redemption            9.3  2948622.0
1                                  The Godfather            9.2  2055335.0
2                                The Dark Knight            9.0  2929338.0
3  The Lord of the Rings: The Return of the King            9.0  2018311.0
4                                Schindlers List            9.0  1479154.0


Após o uso de 'numVotes' > 100.000 , os filmes melhores avaliados são:
1. **The Shawshank Redemption** (Nota: 9.3)
2. **The Godfather** (Nota: 9.2)
3. **The Dark Knight** (Nota: 9.0)
4. **The Lord of the Rings: The Return of the King** (Nota: 9.0)
5. **Schindlers List** (Nota: 9.0)

# Qual é o gênero mais frequente entre os filmes com nota maior que 8?

### Analisando os gêneros presentes na tabela

In [6]:
query = """
SELECT DISTINCT genres
FROM basics;
"""
generos_basics = pd.read_sql_query(query, conn)
print(generos_basics)

                            genres
0                Documentary,Short
1                  Animation,Short
2         Animation,Comedy,Romance
3                     Comedy,Short
4                            Short
...                            ...
2376          Adult,Short,Thriller
2377     Adult,Animation,Talk-Show
2378       Biography,Crime,Fantasy
2379   Mystery,Reality-TV,Thriller
2380  Musical,Reality-TV,Talk-Show

[2381 rows x 1 columns]


Buscando gênero mais frequente em notas maiores que 8:

In [8]:
high_rating = pd.read_sql_query("""
SELECT genres 
FROM basics b
JOIN ratings r ON b.tconst = r.tconst
WHERE r.averageRating > 8
AND b.titleType = 'movie'
""", conn)

from collections import Counter

all_genres = Counter()
for g in high_rating['genres'].dropna():
    for genre in g.split(','):
        all_genres[genre] += 1

print(all_genres.most_common(2))

[('Documentary', 10564), ('Drama', 8114)]


Logo, o gênero com maior contagem de notas atribuídas maiores que 8 é **Documentary**. Este fenômeno ocorre porque documentários, com menos avaliações e públicos mais nichados, mantêm notas altas mais facilmente. Enquanto filmes de outros gêneros recebem volume massivo de votos com maior variação, documentários beneficiam-se de engajamento mais favorável entre seu público específico.

# Quais são os 3 atores/atrizes que mais participaram de filmes com nota maior que 7.5?

**Contagem de atores**

In [3]:
query = """
SELECT COUNT(DISTINCT nconst)
FROM principals
WHERE characters IS NOT NULL;
"""
atores = pd.read_sql_query(query, conn)
print(atores)

   COUNT(DISTINCT nconst)
0                 6442349


**Id dos Filmes com averageRating > 7.5**

In [4]:
query = """
SELECT r.tconst
FROM ratings r
JOIN basics b ON r.tconst = b.tconst
WHERE r.averageRating > 7.5
AND b.titleType = 'movie';
"""
filmes = pd.read_sql_query(query, conn)
print(filmes)


          tconst
0      tt0002305
1      tt0003386
2      tt0003748
3      tt0003771
4      tt0004167
...          ...
47495  tt9911774
47496  tt9914642
47497  tt9914644
47498  tt9916538
47499  tt9916706

[47500 rows x 1 columns]


In [5]:
query = """
SELECT nconst , COUNT(tconst) AS Total
FROM principals
WHERE tconst IN (SELECT r.tconst
                 FROM ratings r
                 JOIN basics b ON r.tconst = b.tconst
                 WHERE r.averageRating > 7.5
                 AND b.titleType = 'movie')
GROUP BY nconst
ORDER BY Total DESC 
LIMIT 3
"""

atores = pd.read_sql_query(query, conn)
print(atores)

      nconst  Total
0  nm0004660    233
1  nm5954636    200
2  nm0595934    170


Como não foi fornecido o nome dos atores, foi selecionado os ID's dos 3 autores.
- **nm0004660** (233 participações)
- **nm5954636** (200 participações)
- **nm0595934** (170 participações)

In [16]:
conn.close()